In [ ]:
"""keep reading: we are going to do a bunch of stuff with type hints"""
from __future__ import annotations

docs notebooks
==============

notebooks are used for both static, narrative interactive documentation, as well as interactive experiences.

`importnb` further lets notebooks show up in interlinked api documentation from import-based frameworks.

## writing code for docs

while the specific syntax for embedding api documentation varies based on the framework, python-level conventions such as docstrings and type annotations provide value to both interactive runtime documentation.

> this module uses PEP 484 _type hints_, specifically the more robust explicit annotations feature, which must be the first executable line, and all imports (like `pandas` below) need to resolve

this docstring will be used for ``a_demo_function`` when imported or documented, conventionally intrepreted as ``.rst``.

In [ ]:
def a_demo_function():
    return "demo"

> 😈 live [introspection](https://jupyter-client.readthedocs.io/en/latest/messaging.html#introspection) won't respect the above, showing this as having no docstring

## a demo class 

because `DemoClass` provides a docstring (conventionally, in `.rst`), this markdown cell will not be used.

using type hints help create rich, interlinked documentation from machine-readable data, explored further below in [read docs data](#read-docs-data).

In [ ]:
from dataclasses import dataclass


@dataclass
class DemoClass:
    """the documentation for the class, written in ``.rst``."""

    #: members can have above-line docs, can use simple ``.rst``
    foo: int

    bar: str = "bar"
    """members can use a multi-line `.rst` docstring...

    * subsequent lines are omitted, by default
    """

    def baz() -> bool:
        """Method members generally use the multiline pattern.

        Code blocks `will` be expanded here:

        .. code:

            DemoClass().baz()

        """
        return True

## build docs

this build uses ``sphinx``, which can make a single html file from any number of source documents, usually ``.rst``. ``myst-parser`` expands this to include ``.md`` files.

In [ ]:
def build_docs() -> bytes:
    import tempfile
    from pathlib import Path

    from sphinx.cmd.build import build_main

    import importnb

    with importnb.Notebook(), tempfile.TemporaryDirectory() as td:
        from lite_helpers import sphinx_html

        build_main(["-qEb", "singlehtml", ".", f"{td}/html"])
        sphinx_html(f"{td}/html")
        objects_inv_bytes = Path(f"{td}/html/objects.inv").read_bytes()
    return objects_inv_bytes

In [ ]:
if __name__ == "__main__":
    # 💡 not using `pandas` just yet, but all hints need to resolve
    %pip install myst-parser sphinx-autodoc-typehints pandas
    objects_inv_bytes = build_docs()

## read docs data

the `objects.inv` data captured above shows the well-identified symbols that would be available to many other sites built on `sphinx` via [intersphinx](https://www.sphinx-doc.org/en/master/usage/extensions/intersphinx.html), or even `mkdocs` via [`mkdocstrings`](https://github.com/mkdocstrings/mkdocstrings?tab=readme-ov-file#quick-usage). 

> 💡 this demo does _not_ use `intersphinx`, which would be neat, as the above docs would link to the python docs. however, the implementation has a difficult-to-patch [use of threads](https://github.com/sphinx-doc/sphinx/blob/v8.2.3/sphinx/ext/intersphinx/_load.py#L175), which are [not yet supported in pyodide](https://github.com/pyodide/pyodide/issues/237).

In [ ]:
import typing

if typing.TYPE_CHECKING:
    # `pandas` pulls a _lot_ of data
    from pandas import DataFrame


def read_docs_data(objects_inv_bytes: bytes) -> DataFrame:
    """Read the ``objects.inv`` data"""
    import tempfile
    from pathlib import Path

    import pandas as pd
    from sphobjinv import Inventory

    with tempfile.TemporaryDirectory() as td:
        objects_inv = Path(td) / "objects.inv"
        objects_inv.write_bytes(objects_inv_bytes)
        inv = Inventory(objects_inv)
        inv_dict = inv.json_dict()
        [inv_dict.pop(m) for m in ["project", "version", "count"]]
        df = pd.DataFrame(
            inv_dict,
        )
        display(df.T)
    return df

In [ ]:
if __name__ == "__main__":
    %pip install sphobjinv ssl pandas
    df = read_docs_data(objects_inv_bytes)